In [2]:
%conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda




In [3]:
%pip install torch torchvision matplotlib pillow

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import torch
import torchvision
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image, make_grid
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from glob import glob

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Settings
image_size = 256
batch_size = 64
latent_dim = 100
epochs = 100
lr = 0.0002
beta1 = 0.5
save_dir = "./generated"
os.makedirs(save_dir, exist_ok=True)

DATASET_ROOT = "C:/College/Projects/X-RayComparison/Data/train/"
XR_TYPE = "XR_ELBOW"  # or XR_ELBOW, XR_HAND, etc.
IMG_DIR = os.path.join(DATASET_ROOT, XR_TYPE)


In [2]:
!nvidia-smi

Sun Jun  8 21:38:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   52C    P5              8W /  120W |     924MiB /   8188MiB |     13%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 4070 Laptop GPU


In [7]:
import os
from glob import glob
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as T

class MURAXrayDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        # Example: root_dir = "C:/College/Projects/X-RayComparison/Data/train"
        self.image_paths = glob(os.path.join(root_dir, "**", "*.png"), recursive=True)
        if len(self.image_paths) == 0:
            raise ValueError(f"No .png images found in path: {root_dir}")
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")  # Convert to grayscale
        if self.transform:
            image = self.transform(image)
        return image


In [8]:
print("IMG_DIR:", IMG_DIR)
print("Folders inside:", os.listdir(IMG_DIR))

IMG_DIR: C:/College/Projects/X-RayComparison/Data/train/XR_ELBOW
Folders inside: ['patient00011', 'patient00016', 'patient00026', 'patient00031', 'patient00034', 'patient00044', 'patient00069', 'patient00147', 'patient00148', 'patient00164', 'patient00180', 'patient00196', 'patient00222', 'patient00238', 'patient00266', 'patient00280', 'patient00310', 'patient00328', 'patient00354', 'patient00357', 'patient00370', 'patient00374', 'patient00377', 'patient00380', 'patient00398', 'patient00401', 'patient00404', 'patient00427', 'patient00434', 'patient00463', 'patient00466', 'patient00511', 'patient00537', 'patient00562', 'patient00643', 'patient00646', 'patient00665', 'patient00673', 'patient00707', 'patient00737', 'patient00745', 'patient00821', 'patient00876', 'patient00894', 'patient00901', 'patient00905', 'patient00918', 'patient00936', 'patient00956', 'patient00982', 'patient00990', 'patient01005', 'patient01055', 'patient01057', 'patient01079', 'patient01112', 'patient01114', 'patie

In [9]:
# Your dataset root
IMG_DIR = "C:/College/Projects/X-RayComparison/Data/train"

# Your transform
transform = T.Compose([
    T.Resize((128, 128)),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,))
])

# Initialize dataset and dataloader
dataset = MURAXrayDataset(IMG_DIR, transform=transform)
print(f"Found {len(dataset)} images")

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


Found 36808 images


In [ ]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=1, features_g=64):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # Input: N x z_dim x 1 x 1
            nn.ConvTranspose2d(z_dim, features_g * 8, kernel_size=4, stride=1, padding=0),  # 4x4
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g * 8, features_g * 4, kernel_size=4, stride=2, padding=1),  # 8x8
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g * 4, features_g * 2, kernel_size=4, stride=2, padding=1),  # 16x16
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g * 2, features_g, kernel_size=4, stride=2, padding=1),  # 32x32
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),

            nn.ConvTranspose2d(features_g, img_channels, kernel_size=4, stride=2, padding=1),  # 64x64
            nn.Tanh()  # Output in range [-1, 1]
        )

    def forward(self, x):
        return self.gen(x)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, img_channels=1, features_d=64):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # Input: N x 1 x 64 x 64
            nn.Conv2d(img_channels, features_d, kernel_size=4, stride=2, padding=1),  # 32x32
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features_d, features_d * 2, kernel_size=4, stride=2, padding=1),  # 16x16
            nn.BatchNorm2d(features_d * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features_d * 2, features_d * 4, kernel_size=4, stride=2, padding=1),  # 8x8
            nn.BatchNorm2d(features_d * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features_d * 4, features_d * 8, kernel_size=4, stride=2, padding=1),  # 4x4
            nn.BatchNorm2d(features_d * 8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=1, padding=0),  # 1x1
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.disc(x).view(-1)


In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Hyperparameters
z_dim = 100
lr = 2e-6
beta1 = 0.5
device = "cuda" if torch.cuda.is_available() else "cpu"

gen = Generator(z_dim).to(device)
disc = Discriminator().to(device)
gen.apply(weights_init)
disc.apply(weights_init)

criterion = nn.BCELoss()
opt_gen = torch.optim.Adam(gen.parameters(), lr=lr, betas=(beta1, 0.999))
opt_disc = torch.optim.Adam(disc.parameters(), lr=lr, betas=(beta1, 0.999))


In [35]:
%pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [38]:
from tqdm import tqdm
import torch

epochs = 100
fixed_noise = torch.randn(64, z_dim, 1, 1).to(device)

# Store previous losses
prev_lossG = float('inf')
bad_gen_epochs = 0

for epoch in range(epochs):
    loop = tqdm(dataloader, desc=f"Epoch [{epoch+1}/{epochs}]")
    for batch in loop:
        real = batch.to(device)
        noise = torch.randn(real.size(0), z_dim, 1, 1).to(device)
        fake = gen(noise)

        ### Train Discriminator ###
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))

        disc_fake = disc(fake.detach()).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))

        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        ### Train Generator ###
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        # Feedback loop: train generator again if struggling
        if lossG.item() > lossD.item():
            gen.zero_grad()
            noise2 = torch.randn(real.size(0), z_dim, 1, 1).to(device)
            fake2 = gen(noise2)
            output2 = disc(fake2).view(-1)
            lossG2 = criterion(output2, torch.ones_like(output2))
            lossG2.backward()
            opt_gen.step()
            lossG = (lossG + lossG2) / 2  # average loss after 2 rounds

        loop.set_postfix(lossD=lossD.item(), lossG=lossG.item())

    print(f"Epoch [{epoch+1}/{epochs}] Loss D: {lossD:.4f}, Loss G: {lossG:.4f}")

    # Track consecutive bad epochs for generator
    if lossG.item() > prev_lossG:
        bad_gen_epochs += 1
    else:
        bad_gen_epochs = 0

    prev_lossG = lossG.item()

    # Optional: Early intervention if gen performs poorly for many epochs
    if bad_gen_epochs >= 5:
        print("Warning: Generator struggling for 5+ epochs. Consider lowering LR or adjusting model.")


Epoch [1/100]: 100%|██████████| 576/576 [01:42<00:00,  5.62it/s, lossD=5.66e-8, lossG=19.9]


Epoch [1/100] Loss D: 0.0000, Loss G: 19.9475


Epoch [2/100]: 100%|██████████| 576/576 [01:49<00:00,  5.27it/s, lossD=6.39e-7, lossG=16.1]


Epoch [2/100] Loss D: 0.0000, Loss G: 16.0763


Epoch [3/100]: 100%|██████████| 576/576 [01:49<00:00,  5.27it/s, lossD=7.16e-8, lossG=17.5]


...

Epoch [98/100] Loss D: 0.0000, Loss G: 21.4437


Epoch [99/100]: 100%|██████████| 576/576 [01:55<00:00,  4.98it/s, lossD=4.76e-10, lossG=21.8]


Epoch [99/100] Loss D: 0.0000, Loss G: 21.7716


Epoch [100/100]: 100%|██████████| 576/576 [01:52<00:00,  5.10it/s, lossD=3.81e-10, lossG=21.3]

Epoch [100/100] Loss D: 0.0000, Loss G: 21.3399


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

# -------------------------
# 1. Hyperparameters
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
z_dim = 100
image_size = 64
channels_img = 1  # for grayscale MURA images
features_gen = 64
features_disc = 64
lr = 2e-6
batch_size = 128
epochs = 100
beta1 = 0.5
beta2 = 0.999

# -------------------------
# 2. Weight Initialization
# -------------------------
def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
        nn.init.normal_(m.weight.data, 0.0, 0.02)

# -------------------------
# 3. Generator
# -------------------------
class Generator(nn.Module):
    def __init__(self, z_dim, channels_img, features_g):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z_dim, features_g * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 8, features_g * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 4, features_g * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 2, features_g, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g, channels_img, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.net(x)

# -------------------------
# 4. Discriminator
# -------------------------
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(channels_img, features_d, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d, features_d * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d * 2, features_d * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d * 4, features_d * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.net(x)

# -------------------------
# 5. Initialize Models
# -------------------------
gen = Generator(z_dim, channels_img, features_gen).to(device)
disc = Discriminator(channels_img, features_disc).to(device)
gen.apply(weights_init)
disc.apply(weights_init)

# -------------------------
# 6. Optimizers and Loss
# -------------------------
criterion = nn.BCELoss()
opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=(beta1, beta2))
opt_disc = optim.Adam(disc.parameters(), lr=lr, betas=(beta1, beta2))

# -------------------------
# 7. Dataset & Loader (adjust root to your dataset)
# -------------------------
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

from torchvision.datasets import ImageFolder
from PIL import Image, UnidentifiedImageError

class SafeImageFolder(ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            sample = self.loader(path)
        except (UnidentifiedImageError, OSError):
            # Return a dummy image (black) and ignore this index
            sample = Image.new('L', (image_size, image_size))  # Grayscale fallback
            target = -1
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target


dataset = SafeImageFolder(root="C:/College/Projects/X-RayComparison/Data/train", transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)


fixed_noise = torch.randn(64, z_dim, 1, 1).to(device)




In [ ]:
# -------------------------
# 8. Training Loop
# -------------------------
for epoch in range(epochs):
    loop = tqdm(dataloader, leave=True)
    for i, (real, _) in enumerate(loop):
        real = real.to(device)
        noise = torch.randn(real.size(0), z_dim, 1, 1).to(device)
        fake = gen(noise)

        ### Train Discriminator ###
        disc_real = disc(real).view(-1)
        label_real = torch.full_like(disc_real, 0.9)  # label smoothing
        lossD_real = criterion(disc_real, label_real)

        disc_fake = disc(fake.detach()).view(-1)
        label_fake = torch.zeros_like(disc_fake)
        lossD_fake = criterion(disc_fake, label_fake)

        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        ### Train Generator ###
        output = disc(fake).view(-1)
        label_gen = torch.ones_like(output)  # try to fool D
        lossG = criterion(output, label_gen)
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
        loop.set_postfix(lossD=lossD.item(), lossG=lossG.item())

    # Save samples
    if (epoch + 1) % 5 == 0:
        with torch.no_grad():
            fake_samples = gen(fixed_noise).detach().cpu()
            grid = make_grid(fake_samples, normalize=True)
            save_image(grid, f"samples/fake_epoch_{epoch+1}.png")

    print(f"Epoch [{epoch+1}/{epochs}] Loss D: {lossD.item():.4f}, Loss G: {lossG.item():.4f}")

  0%|          | 0/288 [00:00<?, ?it/s]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from tqdm import tqdm
import os

# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
z_dim = 100
batch_size = 64
lr = 2e-4
image_size = 128
channels_img = 1  # grayscale
features_gen = 64
features_disc = 64
epochs = 100
critic_iter = 1  # how often to train D per G update

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])

dataset = datasets.ImageFolder(root='C:/College/Projects/X-RayComparison/Data/train', transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Generator
class Generator(nn.Module):
    def __init__(self, z_dim, channels_img, features_g):
        super().__init__()
        self.gen = nn.Sequential(
            self._block(z_dim, features_g * 16, 4, 1, 0),   # 1x1 -> 4x4
            self._block(features_g * 16, features_g * 8, 4, 2, 1),  # 4x4 -> 8x8
            self._block(features_g * 8, features_g * 4, 4, 2, 1),   # 8x8 -> 16x16
            self._block(features_g * 4, features_g * 2, 4, 2, 1),   # 16x16 -> 32x32
            self._block(features_g * 2, features_g, 4, 2, 1),       # 32x32 -> 64x64
            nn.ConvTranspose2d(features_g, channels_img, 4, 2, 1),  # 64x64 -> 128x128
            nn.Tanh()
        )

    def _block(self, in_c, out_c, k, s, p):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, k, s, p, bias=False),
            nn.BatchNorm2d(out_c),
            nn.ReLU(True)
        )

    def forward(self, x):
        return self.gen(x)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super().__init__()
        self.disc = nn.Sequential(
            self._block(channels_img, features_d, 4, 2, 1),       # 128 -> 64
            self._block(features_d, features_d * 2, 4, 2, 1),     # 64 -> 32
            self._block(features_d * 2, features_d * 4, 4, 2, 1), # 32 -> 16
            self._block(features_d * 4, features_d * 8, 4, 2, 1), # 16 -> 8
            self._block(features_d * 8, features_d * 16, 4, 2, 1),# 8 -> 4
            nn.Conv2d(features_d * 16, 1, 4, 1, 0),               # 4 -> 1
            nn.Sigmoid()
        )

    def _block(self, in_c, out_c, k, s, p):
        return nn.Sequential(
            nn.Conv2d(in_c, out_c, k, s, p, bias=False),
            nn.BatchNorm2d(out_c),
            nn.LeakyReLU(0.2, inplace=True)
        )

    def forward(self, x):
        return self.disc(x).view(-1)

# Initialize
gen = Generator(z_dim, channels_img, features_gen).to(device)
disc = Discriminator(channels_img, features_disc).to(device)
opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))
criterion = nn.BCELoss()
fixed_noise = torch.randn(64, z_dim, 1, 1).to(device)



In [ ]:
# Training Loop
for epoch in range(epochs):
    loop = tqdm(dataloader, leave=True)
    for i, (real, _) in enumerate(loop):
        real = real.to(device)
        noise = torch.randn(real.size(0), z_dim, 1, 1).to(device)
        fake = gen(noise)

        # Train Discriminator
        disc_real = disc(real)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real) * 0.9)  # Label smoothing

        disc_fake = disc(fake.detach())
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))

        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        # Train Generator
        if i % critic_iter == 0:
            output = disc(fake)
            lossG = criterion(output, torch.ones_like(output))  # Generator wants D to output 1
            gen.zero_grad()
            lossG.backward()
            opt_gen.step()

        loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
        loop.set_postfix(lossD=lossD.item(), lossG=lossG.item())

    # Save samples
    if (epoch + 1) % 10 == 0:
        utils.save_image(gen(fixed_noise), f"generated_epoch_{epoch+1}.png", normalize=True)



Epoch [1/100]:   9%|▉         | 51/576 [00:25<04:21,  2.01it/s, lossD=0.178, lossG=8.3] 


UnidentifiedImageError: cannot identify image file <_io.BufferedReader name='C:/College/Projects/X-RayComparison/Data/train\\XR_WRIST\\patient07840\\study2_negative\\._image3.png'>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
import os
from tqdm import tqdm

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = 128
z_dim = 128
batch_size = 64
lambda_gp = 10
num_epochs = 100
lr = 1e-4

# Transforms
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])

# Dataset
dataset_path = "C:/College/Projects/X-RayComparison/Data/train"  # Update this to the correct path
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

# Generator
class Generator(nn.Module):
    def __init__(self, z_dim=128, img_channels=1, features_g=64):
        super().__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(z_dim, features_g*16, 4, 1, 0),  # 4x4
            nn.BatchNorm2d(features_g*16),
            nn.ReLU(),

            nn.ConvTranspose2d(features_g*16, features_g*8, 4, 2, 1),  # 8x8
            nn.BatchNorm2d(features_g*8),
            nn.ReLU(),

            nn.ConvTranspose2d(features_g*8, features_g*4, 4, 2, 1),  # 16x16
            nn.BatchNorm2d(features_g*4),
            nn.ReLU(),

            nn.ConvTranspose2d(features_g*4, features_g*2, 4, 2, 1),  # 32x32
            nn.BatchNorm2d(features_g*2),
            nn.ReLU(),

            nn.ConvTranspose2d(features_g*2, features_g, 4, 2, 1),  # 64x64
            nn.BatchNorm2d(features_g),
            nn.ReLU(),

            nn.ConvTranspose2d(features_g, img_channels, 4, 2, 1),  # 128x128
            nn.Tanh()
        )

    def forward(self, x):
        return self.gen(x)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, img_channels=1, features_d=64):
        super().__init__()
        def block(in_c, out_c, k, s, p):
            return nn.Sequential(
                nn.utils.spectral_norm(nn.Conv2d(in_c, out_c, k, s, p)),
                nn.LayerNorm([out_c, image_size // s, image_size // s]),
                nn.LeakyReLU(0.2)
            )

        self.disc = nn.Sequential(
            block(img_channels, features_d, 4, 2, 1),    # 64
            block(features_d, features_d*2, 4, 2, 1),     # 32
            block(features_d*2, features_d*4, 4, 2, 1),   # 16
            block(features_d*4, features_d*8, 4, 2, 1),   # 8
            block(features_d*8, features_d*16, 4, 2, 1),  # 4
            nn.Conv2d(features_d*16, 1, 4, 1, 0),         # 1
        )

    def forward(self, x):
        return self.disc(x).view(-1)

# Gradient Penalty
def gradient_penalty(critic, real, fake):
    batch_size, C, H, W = real.shape
    epsilon = torch.rand(batch_size, 1, 1, 1, device=device).expand_as(real)
    interpolated = (epsilon * real + (1 - epsilon) * fake).requires_grad_(True)

    mixed_scores = critic(interpolated)
    gradient = grad(
        outputs=mixed_scores,
        inputs=interpolated,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(batch_size, -1)
    gp = ((gradient.norm(2, dim=1) - 1) ** 2).mean()
    return gp

# Initialize
gen = Generator(z_dim=z_dim).to(device)
critic = Discriminator().to(device)

opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=(0.0, 0.9))
opt_critic = optim.Adam(critic.parameters(), lr=lr, betas=(0.0, 0.9))


In [11]:
import torch
import torch.nn as nn

class Critic(nn.Module):
    def __init__(self, img_channels=1, feature_d=64):
        super(Critic, self).__init__()
        self.model = nn.Sequential(
            # Input: N x 1 x 128 x 128
            nn.Conv2d(img_channels, feature_d, kernel_size=4, stride=2, padding=1),  # 64x64
            nn.LeakyReLU(0.2),

            nn.Conv2d(feature_d, feature_d * 2, kernel_size=4, stride=2, padding=1),  # 32x32
            nn.BatchNorm2d(feature_d * 2),
            nn.LeakyReLU(0.2),

            nn.Conv2d(feature_d * 2, feature_d * 4, kernel_size=4, stride=2, padding=1),  # 16x16
            nn.BatchNorm2d(feature_d * 4),
            nn.LeakyReLU(0.2),

            nn.Conv2d(feature_d * 4, feature_d * 8, kernel_size=4, stride=2, padding=1),  # 8x8
            nn.BatchNorm2d(feature_d * 8),
            nn.LeakyReLU(0.2),

            nn.Conv2d(feature_d * 8, feature_d * 16, kernel_size=4, stride=2, padding=1),  # 4x4
            nn.BatchNorm2d(feature_d * 16),
            nn.LeakyReLU(0.2),

            nn.Conv2d(feature_d * 16, 1, kernel_size=4, stride=1, padding=0),  # 1x1
        )

    def forward(self, x):
        return self.model(x).view(-1)

# Example instantiation
critic = Critic(img_channels=1).to("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from PIL import Image, UnidentifiedImageError

class SafeImageFolder(ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            sample = self.loader(path)
        except UnidentifiedImageError:
            # Replace with a black image or skip with random image
            sample = Image.new("L", (128, 128))  # Grayscale fallback
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

dataset = SafeImageFolder(root="C:/College/Projects/X-RayComparison/Data/train", transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4)


In [ ]:
import torchvision

critic_iterations = 5  # Number of updates for the discriminator per generator update
step = 0
fixed_noise = torch.randn(64, z_dim, 1, 1).to(device)

for epoch in range(num_epochs):
    loop = tqdm(dataloader, leave=True)
    for batch_idx, (real, _) in enumerate(loop):
        real = real.to(device)
        cur_batch_size = real.size(0)

        ### Train Critic ###
        for _ in range(critic_iterations):
            noise = torch.randn(cur_batch_size, z_dim, 1, 1).to(device)
            fake = gen(noise)

            critic_real = critic(real).reshape(-1)
            critic_fake = critic(fake.detach()).reshape(-1)
            gp = gradient_penalty(critic, real, fake)
            loss_critic = -(torch.mean(critic_real) - torch.mean(critic_fake)) + lambda_gp * gp

            opt_critic.zero_grad()
            loss_critic.backward()
            opt_critic.step()

        ### Train Generator ###
        noise = torch.randn(cur_batch_size, z_dim, 1, 1).to(device)
        fake = gen(noise)
        output = critic(fake).reshape(-1)
        loss_gen = -torch.mean(output)

        opt_gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        # Logging
        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(critic_loss=loss_critic.item(), gen_loss=loss_gen.item())

        # Save sample generated images
        if step % 500 == 0:
            with torch.no_grad():
                fake = gen(fixed_noise)
                utils.save_image(fake, f"generated_samples/sample_{step}.png", normalize=True)
        step += 1

    # Optional: Save model checkpoints every few epochs
    if (epoch + 1) % 10 == 0:
        torch.save(gen.state_dict(), f"checkpoints/gen_epoch_{epoch+1}.pth")
        torch.save(critic.state_dict(), f"checkpoints/critic_epoch_{epoch+1}.pth")


  0%|          | 0/576 [00:00<?, ?it/s]

In [ ]:
# Training Loop
fixed_noise = torch.randn(64, z_dim, 1, 1).to(device)

for epoch in range(num_epochs):
    for i, (real, _) in enumerate(tqdm(dataloader)):
        real = real.to(device)
        cur_batch_size = real.size(0)

        # Train Critic
        for _ in range(5):
            noise = torch.randn(cur_batch_size, z_dim, 1, 1).to(device)
            fake = gen(noise)

            critic_real = critic(real)
            critic_fake = critic(fake.detach())
            gp = gradient_penalty(critic, real, fake)
            loss_critic = -(torch.mean(critic_real) - torch.mean(critic_fake)) + lambda_gp * gp

            opt_critic.zero_grad()
            loss_critic.backward()
            opt_critic.step()

        # Train Generator
        noise = torch.randn(cur_batch_size, z_dim, 1, 1).to(device)
        fake = gen(noise)
        output = critic(fake)
        loss_gen = -torch.mean(output)

        opt_gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

    print(f"Epoch [{epoch+1}/{num_epochs}] | Loss D: {loss_critic:.4f} | Loss G: {loss_gen:.4f}")
    if (epoch + 1) % 10 == 0:
        utils.save_image(fake[:64], f"samples/wgan_gp_{epoch+1}.png", normalize=True)


NameError: name 'torch' is not defined

In [ ]:
# Load and show last generated image
from torchvision.io import read_image
last_image = read_image(f"{save_dir}/epoch_{epochs}.png").float() / 255
show_image(last_image)

Legacy Cells for reference (Not Working)

In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

class XrayDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.files = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith(".png")]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image = Image.open(self.files[idx])
        if self.transform:
            image = self.transform(image)
        return image

dataset = XrayDataset("C:/College/Projects/X-RayComparison/Data/train", transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as T

transform = T.Compose([
    T.Resize((64, 64)),     # You can use (128, 128) if GPU allows
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,))
])

dataset = MURAXrayDataset(IMG_DIR, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

for epoch in range(epochs):
    for i, real_images in enumerate(dataloader):
        real_images = real_images.to(device)
        batch_size = real_images.size(0)

        # Discriminator
        noise = torch.randn(batch_size, latent_dim, 1, 1, device=device)
        fake_images = generator(noise)

        real_labels = torch.ones(batch_size, device=device)
        fake_labels = torch.zeros(batch_size, device=device)

        output_real = discriminator(real_images)
        loss_real = criterion(output_real, real_labels)

        output_fake = discriminator(fake_images.detach())
        loss_fake = criterion(output_fake, fake_labels)

        loss_D = loss_real + loss_fake
        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

        # Generator
        output = discriminator(fake_images)
        loss_G = criterion(output, real_labels)
        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        if i % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}] Batch {i}/{len(dataloader)} | Loss D: {loss_D:.4f}, Loss G: {loss_G:.4f}")

    # Save generated images after each epoch
    with torch.no_grad():
        fake = generator(fixed_noise).detach().cpu()
    grid = make_grid(fake, padding=2, normalize=True)
    save_image(grid, f"{save_dir}/epoch_{epoch+1}.png")


ValueError: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([1600])) is deprecated. Please ensure they have the same size.

In [ ]:
from tqdm import tqdm

epochs = 100
fixed_noise = torch.randn(64, z_dim, 1, 1).to(device)

for epoch in range(epochs):
    for batch in tqdm(dataloader):
        real = batch.to(device)
        noise = torch.randn(real.size(0), z_dim, 1, 1).to(device)
        fake = gen(noise)

        ### Train Discriminator ###
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))

        disc_fake = disc(fake.detach()).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))

        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        ### Train Generator ###
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))  # Trick D into thinking fake is real
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

    print(f"Epoch [{epoch+1}/{epochs}] Loss D: {lossD:.4f}, Loss G: {lossG:.4f}")

    # Optional: save sample images here using torchvision.utils.save_image


100%|██████████| 576/576 [02:50<00:00,  3.38it/s]


Epoch [1/100] Loss D: 0.0002, Loss G: 9.2120


100%|██████████| 576/576 [02:03<00:00,  4.66it/s]


Epoch [2/100] Loss D: 0.0004, Loss G: 10.0122


100%|██████████| 576/576 [01:26<00:00,  6.67it/s]


...

Epoch [40/100] Loss D: 0.0000, Loss G: 16.3455


100%|██████████| 576/576 [01:15<00:00,  7.60it/s]


Epoch [41/100] Loss D: 0.0000, Loss G: 15.6286


 74%|███████▍  | 427/576 [01:07<00:23,  6.29it/s]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def show_image(tensor_img):
    img = tensor_img.permute(1, 2, 0).squeeze().numpy()
    plt.imshow((img + 1) / 2, cmap='gray')
    plt.axis('off')
    plt.show()

# Load and show last generated image
from torchvision.io import read_image
last_image = read_image(f"{save_dir}/epoch_{epochs}.png").float() / 255
show_image(last_image)


RuntimeError: [Errno 2] No such file or directory: './generated/epoch_100.png'